In [ ]:
import os, sys
import bokeh.models as bokeh_models
import pyarrow as pa

sys.path.append(os.path.abspath('../../src')) # Ideally ST_Visions will be installed as a module, therefore this is more of a development setting for quick testing.

from st_visions.visualization.st_visualizer import st_visualizer
from st_visions.streaming.st_vizstream import ST_KafkaStream

%load_ext autoreload
%autoreload 2
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "*" # DEV ENVIROMENT BANDAID FOR VSCODE VISUALIZATIONS, will be edited

In [ ]:
expected_schema = pa.schema([
    ("lon", pa.float64()),
    ("lat", pa.float64()),
    ("vessel_id", pa.int64()),
    ("speed", pa.float32()),
    ("course", pa.float32()),
    ("heading", pa.float32()),
    ("t", pa.timestamp('ms'))
])

#TODO: set up suffix internally with the cds
st_viz = st_visualizer(limit=5000, expected_schema=expected_schema) # Initialize a VISIONS Instance (ST Visualizer Object)
st_viz.create_canvas(title=f'Showing Streaming Data', tile_provider="CARTODBPOSITRON", sizing_mode='scale_width', height=540, tools="pan, box_zoom, lasso_select, wheel_zoom, hover, save, reset")
circ = st_viz.add_marker(marker='circle', size=10, color='royalblue', alpha=0.7, fill_alpha=0.5, muted_alpha=0, legend_label=f'Vessel GPS Locations')

In [ ]:
stream = ST_KafkaStream(topic_name='st-viz-topic')
st_viz.get_data_stream(stream=stream, notebook=True, refresh_rate=50)

In [ ]:
tooltips = [('Vessel ID','@vessel_id'), ('Timestamp','@t'), ('Speed (knots)','@speed'),
            ('Course over Ground (degrees)','@course'), ('Heading (degrees)','@heading'), ('Coordinates','(@lon_merc, @lat_merc)')]

st_viz.add_hover_tooltips(tooltips)
st_viz.add_lasso_select()


st_viz.figure.legend.location = "top_left"
st_viz.figure.legend.click_policy = "mute"
st_viz.figure.toolbar.active_scroll = st_viz.figure.select_one(bokeh_models.WheelZoomTool)

st_viz.show_figures(notebook=True, height=300, width=300, sizing_mode='stretch_both')